In [4]:
from dotenv import load_dotenv
import os

load_dotenv()  # 自动查找当前目录的 .env 文件

dashscope_api_key = os.getenv("DASHSCOPE_API_KEY")
dashscope_base_url = os.getenv("DASHSCOPE_BASE_URL")

In [5]:
from langchain_qwq import ChatQwen

model = ChatQwen(
    model="qwen-plus",
    api_key=dashscope_api_key,
    base_url=dashscope_base_url,
    temperature=0.7,
)

在实际应用中，我们希望模型可以按照我们指定的格式来提供响应，这有助于输出能够被轻松解析，并用于后续的处理流程。LangChain 支持多种模式类型以及强制执行结构化输出的方法。


# 定义输出结构

当前主流的大模型在进行结构化输出或函数调用（Function Calling）时，其底层通信机制通常就是基于JSON Schema的，因此最底层定义输出结构的方法就是编写一个JSON Schema。

In [1]:
json_schema = {
    "title": "Movie",
    "description": "A movie with details",
    "type": "object",
    "properties": {
        "title": {
            "type": "string",
            "description": "The title of the movie"
        },
        "year": {
            "type": "integer",
            "description": "The year the movie was released"
        },
        "director": {
            "type": "string",
            "description": "The director of the movie"
        },
        "rating": {
            "type": "number",
            "description": "The movie's rating out of 10"
        }
    },
    "required": ["title", "year", "director", "rating"]
}

但在Python环境下，我们通常不直接使用JSON Schema，而是使用Python的TypedDict、Pydantic库或者dataclass来定义输出结构，这比写JSON Schema要方便得多。这三个库都能够定义输出结构，并且都能够进行数据验证，选择哪个主要取决于你的具体使用场景：


1. Pydantic
    - 特点：功能最强大，支持数据验证、默认值、复杂的数据解析（如将字符串自动转为日期对象），<span style="color:red;">会在你创建模型实例时自动校验数据，不符合就抛出错误。</span>
    - 适用场景：
        - 结构化输出：这是 Pydantic 最常见的用法。你可以定义一个 Pydantic 模型，让大模型直接输出符合该模型的 JSON 数据。
        - API 输入/输出：FastAPI 等框架的默认选择，用于验证请求数据。
    - 缺点：引入了额外的依赖，运行时有性能开销。
2. TypedDict
    - 特点：轻量级，仅用于类型提示。它本质上是一个字典，<span style="color:red;">不提供运行时的数据验证（即你传错类型，程序不会报错，只有 IDE 或类型检查器会提示）。</span>
    - 适用场景：
        - 函数参数：当你需要一个类似对象的字典，且希望 IDE 有自动补全提示时。
        - 简单结构：不需要复杂验证，只需要定义键值对的类型。
    - 优点：零运行时开销，代码简洁。
3. Dataclass
    - 特点：主要用于创建可变或不可变的数据容器。它会自动生成 __init__ 等方法。
    - 适用场景：
        - 内部数据结构：在你的应用程序内部，用来组织数据，比如配置项、数据传输对象。
        - 不需要外部验证：如果你不涉及从 JSON 解析数据或不需要严格的外部输入验证，dataclass 非常好用。


In [2]:
from pydantic import BaseModel, Field

class Movie(BaseModel):
    """包含详细信息的电影。"""
    title: str = Field(..., description="电影的标题")
    year: int = Field(..., description="电影的上映年份")
    director: str = Field(..., description="电影的导演")
    rating: float = Field(..., description="电影的评分（满分10分）")

In [ ]:
from typing import TypedDict
from typing_extensions import Annotated

class MovieDict(TypedDict):
    """包含详细信息的电影。"""
    title: Annotated[str, ..., "电影的标题"]
    year: Annotated[int, ..., "电影的上映年份"]
    director: Annotated[str, ..., "电影的导演"]
    rating: Annotated[float, ..., "电影的评分（满分10分）"]

## 结构嵌套

In [ ]:
from pydantic import BaseModel, Field

class Actor(BaseModel):
    """演员信息模型"""
    name: str  # 演员姓名
    role: str  # 扮演角色

class MovieDetails(BaseModel):
    """电影详细信息模型"""
    title: str  # 电影标题
    year: int  # 上映年份
    cast: list[Actor]  # 演员表（包含Actor对象的列表）
    genres: list[str]  # 类型列表（例如：剧情、动作）
    budget: float | None = Field(None, description="预算（单位：百万美元）")  # 制作预算，可为空

# 绑定到模型

In [ ]:
model_with_structure = model.with_structured_output(Movie)
response = model_with_structure.invoke("提供关于电影《盗梦空间》的详细信息")
print(response)

# 其他参数

with_structured_output支持以下参数：
- method：Literal["function_calling", "json_mode", "json_schema"] = "function_calling"
  - 'json_schema' 通常指提供商专门提供的结构化输出功能；
  - 'function_calling' 通过强制模型调用一个符合指定 schema 的工具来生成结构化输出；
  - 'json_mode' 是某些提供商在 'json_schema' 之前提供的一种方式——它能生成有效的 JSON，但 schema 必须在提示（prompt）中进行描述。

  该参数非必需，框架通常会根据你使用的模型自动选择最佳策略。
- Include_raw：bool = False，使用 include_raw=True 可同时获取解析后的输出和原始的 AI 消息。
- strict: Optional[bool] = None，用于控制模型输出是否必须严格符合指定的schema。

In [ ]:
model_with_structure = model.with_structured_output(Movie, include_raw=True)
response = model_with_structure.invoke("提供关于电影《盗梦空间》的详细信息")
print(response)